In [2]:
# Step 1: Install the Kaggle package
!pip install kaggle

# Step 2: Import the required libraries
import os
from google.colab import files

# Step 3: Upload your Kaggle API key file (kaggle.json) that you obtained from your Kaggle account settings
files.upload()  # This will prompt you to select the file from your local machine

# Step 4: Move the uploaded file to the required directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Step 5: Change the permissions of the file
!chmod 600 ~/.kaggle/kaggle.json

# Step 6: Import the Kaggle API library
import kaggle

# Step 7: Use the Kaggle API to download the dataset
# Replace 'kazanova/sentiment140' with the name of the dataset you want to download from Kaggle
# Replace 'path/to/download' with the desired path to download the dataset
kaggle.api.dataset_download_files('kazanova/sentiment140', path='/content', unzip=True)


Saving kaggle.json to kaggle.json


#Dependencies

In [4]:
# Importing necessary libraries
import numpy as np  # NumPy for numerical computing
import pandas as pd  # Pandas for data manipulation
import re  # Regular expression for text preprocessing
from nltk.corpus import stopwords  # NLTK's stopwords for removing common words
from nltk.stem.porter import PorterStemmer  # NLTK's PorterStemmer for word stemming
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF Vectorizer for feature extraction
from sklearn.model_selection import train_test_split  # Train-test split for model evaluation
from sklearn.linear_model import LogisticRegression  # Logistic Regression model for classification
from sklearn.metrics import accuracy_score  # Accuracy score for model evaluation

In [5]:
# Import nltk library
import nltk

# Download the stopwords corpus from nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# Print the list of English stopwords
print(stopwords.words('english'))
#words which don't add any significant meaning to the data

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [8]:
twitter_data.shape

(1599999, 6)

In [9]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
# Define column names for the dataframe
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Read the CSV file into a pandas DataFrame
# Specify the column names and encoding
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding='ISO-8859-1')

In [11]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
# Check for missing values in the dataframe
twitter_data.isnull().sum()


target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [13]:
# Count the frequency of each target value
target_counts = twitter_data['target'].value_counts()
print(target_counts)

0    800000
4    800000
Name: target, dtype: int64

In [14]:
# Replace target values of 4 with 1
twitter_data.replace({'target': {4: 1}}, inplace=True)

In [15]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

##stemming
Stemming is a natural language processing technique used to reduce words to their root or base form, known as the stem. The purpose of stemming is to normalize words so that variations of the same word are treated as identical tokens, thereby reducing redundancy in the text data.

For example, consider the words "running", "runs", and "ran". These words share the same root "run". By stemming, we can convert all these variations to the common stem "run".

In [16]:
# Initializing the Porter Stemmer
port_stem = PorterStemmer()

# Function to perform stemming
def stemming(content):
    # Removing non-alphabetic characters
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    # Converting to lowercase
    stemmed_content = stemmed_content.lower()
    # Splitting into words
    stemmed_content = stemmed_content.split()
    # Stemming and removing stopwords
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Joining the stemmed words
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content


In [17]:
#his will create a new column named 'stemmed_content' containing the stemmed version of the text in each row of the 'text' column.
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming) #almost 50min time taking


In [18]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [19]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [20]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [21]:
# Extracting the preprocessed and stemmed text data from the 'stemmed_content' column
# and assigning it to the variable x
x = twitter_data['stemmed_content'].values

# Extracting the target labels from the 'target' column
# and assigning them to the variable y
y = twitter_data['target'].values

In [22]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [23]:
print(y)

[0 0 0 ... 1 1 1]


# Splitting the data
into training and testing sets using train_test_split function
 x_train: Training data features
 x_test: Testing data features
 y_train: Training data labels
 y_test: Testing data labels
 The test size is set to 20% of the total data
 Stratify parameter is used to ensure that the class distribution is maintained in the train and test sets
 Random state is set to 2 for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)


In [24]:
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2,stratify=y,random_state=2)

In [25]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [26]:
print(y.shape,y_train.shape,y_test.shape)

(1600000,) (1280000,) (320000,)


In [27]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [28]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


# Convert text to numerical data for ML models
The TF-IDF (Term Frequency-Inverse Document Frequency) vectorizer is a feature extraction technique used in natural language processing (NLP) and text mining. It transforms raw text documents into numerical feature vectors, which can then be used as input for machine learning algorithms.

TF-IDF vectorization consists of two main components:

Term Frequency (TF): This component measures the frequency of a term (word) within a document. It is calculated as the number of times a term appears in a document divided by the total number of terms in the document. It helps to highlight the importance of a term within a specific document.

Inverse Document Frequency (IDF): This component measures the importance of a term across the entire corpus (collection of documents). It is calculated as the logarithm of the total number of documents divided by the number of documents containing the term. Terms that appear frequently in many documents have lower IDF scores, while terms that appear rarely have higher IDF scores.


In [29]:
# Creating a TF-IDF vectorizer object
vectorizer = TfidfVectorizer()

# Fitting the vectorizer on the training data and transforming both the training and testing data
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [30]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [31]:
print(x_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

In [ ]:
# Logistic Model use with 1000 iterations

In [32]:
#logistic regression# Importing the Logistic Regression model from scikit-learn
#from sklearn.linear_model import LogisticRegression

# Initializing the Logistic Regression model with specified parameters
# max_iter: Maximum number of iterations for the optimization algorithm to converge
# Setting max_iter to a higher value (e.g., 1000) ensures that the optimization algorithm converges
model = LogisticRegression(max_iter=1000)


In [33]:
# Fitting the Logistic Regression model to the training data
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

#Accuracy

In [34]:
# Making predictions on the training data using the trained model
x_train_prediction = model.predict(x_train)

# Calculating the accuracy of the model on the training data
training_data_accuracy = accuracy_score(y_train, x_train_prediction)

In [35]:
print('Accuracy score on the training data:',training_data_accuracy)

Accuracy score on the training data: 0.81018984375


In [36]:
# Making predictions on the test data using the trained model
x_test_prediction = model.predict(x_test)

# Calculating the accuracy of the model on the test data
testing_data_accuracy = accuracy_score(y_test, x_test_prediction)

In [37]:
print('Accuracy score on the testing data:',testing_data_accuracy)

Accuracy score on the testing data: 0.7780375


Model Accuracy=77.8%(approximately)

# pickle use to save model for future prediction
bcz we don't want to same step trains for predictions

In [38]:
import pickle

In [39]:
# Specify the filename for saving the trained model
filename = 'trained_model.sav'

# Serialize and save the trained model to the specified file
# 'wb' mode is used to open the file in binary write mode
# This allows the model to be written to the file in binary format
pickle.dump(model, open(filename, 'wb'))

In [41]:
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [42]:
# Select a single data point (tweet) from the test set for prediction
x_new = x_test[200]

# Print the actual target value (0 for negative sentiment, 1 for positive sentiment) of the selected tweet
print("Actual Target:", y_test[200])

# Use the trained model to predict the sentiment of the selected tweet
# The prediction result will be either 0 (negative sentiment) or 1 (positive sentiment)
prediction = loaded_model.predict(x_new)

# Print the predicted sentiment of the tweet
print("Predicted Sentiment:", prediction[0])

# Output the interpretation of the prediction
if prediction[0] == 0:
    print('Negative Tweet')
else:
    print('Positive Tweet')

1
[1]
Positive Tweet


In [43]:
x_new=x_test[3]
print(y_test[3])

prediction=loaded_model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet
